In [ ]:
import numpy as np
import pandas as pd

import re
import requests
from bs4 import BeautifulSoup
from src.data.data_wrangling import load_and_clean_movies_df
from src.scripts.scrap_date import *
from src.data.load_data_non_commercial_imdb import *

# Make sure that if any dependencies changes it will be reflected in the notebook (From the ML course)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
df_movies, _, _, _ = load_and_clean_movies_df()

In [13]:
df_mapping_imdb_id_wikipedia_id = load_imdb_id_wikipedia_id(df_movies)

/Users/guillaumevitalis/Documents/EPFL/Master/MA1/Applied data analysis/Project/Project/src/data/load_data_non_commercial_imdb.py:15: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep="\t")


In [14]:
merged_data = pd.merge(df_movies, df_mapping_imdb_id_wikipedia_id, on='wikipedia_ID', how='inner')

In [ ]:
merged_data

,wikipedia_ID,freebase_ID,name,revenue,runtime,release_year,release_month,release_day,plot,tconst
0,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,2001,8,24,"Set in the second half of the 22nd century, th...",tt0228333
1,9363483,/m/0285_cd,White Of The Eye,NaN,110.0,1987,<NA>,<NA>,A series of murders of rich young women throug...,tt0094320
2,261236,/m/01mrr1,A Woman in Flames,NaN,106.0,1983,<NA>,<NA>,"Eva, an upper class housewife, becomes frustra...",tt0083949
3,6631279,/m/0gffwj,Little city,NaN,93.0,1997,4,4,"Adam, a San Francisco-based artist who works a...",tt0119548
4,171005,/m/016ywb,Henry V,10161099.0,137.0,1989,11,8,{{Plot|dateAct 1Act 2Act 3Act 4Act 5 Finally n...,tt0097499
...,...,...,...,...,...,...,...,...,...,...
28895,3868432,/m/0b44p5,Into the Mirror,NaN,113.0,2003,8,14,After accidentally causing the death of his pa...,tt0372937
28896,15394941,/m/03m6zh4,Gopi Kishan,NaN,NaN,1994,12,2,Kishan killed a man in his childhood when the...,tt0109922
28897,23851782,/m/06_vb43,The Ghost Train,NaN,82.0,1941,5,3,The film opens with a Great Western express sp...,tt0033660
28898,35228177,/m/0j7hxnt,Mermaids: The Body Found,NaN,120.0,2011,3,19,Two former National Oceanic Atmospheric Admini...,tt1816585


In [ ]:
n_total = merged_data.shape[0]
no_dates = merged_data.loc[(merged_data.release_month != '<NA>') & (merged_data.release_year != '<NA>')].shape[0]

print(f"There are {no_dates} movies among {n_total} for which we have the year and month of release.")

There are 18351 movies among 28900 for which we have the year and month of release.


Let's merge the entire dataframe of the movies with the one containing those selected with their IMDB ids in order to work only with this subset of movies.

In [ ]:
merged_data = pd.read_csv('data/merged_data.csv', sep=',')
merged_data = merged_data.rename(columns={"0": "wikipedia_ID", "2": "title"})

merged_data = pd.merge(df_movies, merged_data, on='wikipedia_ID', how='inner')

In [5]:
n_total = merged_data.shape[0]
no_dates = merged_data.loc[(merged_data.release_month != '<NA>') & (merged_data.release_year != '<NA>')].shape[0]

print(f"There are {no_dates} movies among {n_total} for which we have the year and month of release.")

There are 18181 movies among 28415 for which we have the year and month of release.


Let's now use the wikipedia ID we have for each movie to scrap information from wikidata about the release dates.

In [ ]:
# Create a copy of the dataframe of movies
merged_scraped = merged_data.copy()

merged_data = merged_data.rename(columns={"release_year": "release_year_x", "release_month": "release_month_x"})

dates_merged = merged_data[['wikipedia_ID', 'release_year_x', 'release_month_x']].copy()

# add empty columns that are going to be fill by scraping the dates
dates_merged['release_year_y'] = ''
dates_merged['release_month_y'] = ''

In [ ]:
# Save the file where we will be scraping to
dates_merged.tocsv('dates_scraped.csv', index=False)

In [10]:
# Create a variable corresponding to how many movies we scraped, in case we need to
# to re launch the scraping while some data is already scraped.
number_scraped = 0

We have all we need to scrap ! Let's do it :

In [ ]:
# TO RUN ONLY IF DATES_SCRAPED ALREADY EXITS
# If we need to continue to scrap on a file we've already started to scrap to.
dates_merged = pd.read_csv('dates_scraped.csv', sep=',')

In [11]:
for idx, row in dates_merged.iloc[number_scraped:].iterrows():
    w_id = row.wikipedia_ID
    year, month = format_date_numeric(get_release_date(wikidata_from_wikipedia_id(w_id)[1]))
    dates_merged.loc[idx, 'release_year_y'] = year
    dates_merged.loc[idx, 'release_month_y'] = month

    # save the file with the new scraped dates
    dates_merged.to_csv('dates_scraped.csv', index=False)
    number_scraped += 1

NameError: name 'dates_merged' is not defined